In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/PyAI'

/content/drive/MyDrive/PyAI


In [9]:
import torchvision.datasets as datasets #torch에서 데이터 불러오기
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [5]:
print(torch.cuda.get_device_name())

Tesla T4


In [6]:
train = datasets.CIFAR10(root = "./dataset", train = True, transform=transforms.ToTensor(), download=True)
test = datasets.CIFAR10(root = "./dataset", train = False, transform=transforms.ToTensor(), download=True)

train_data = DataLoader(train, batch_size = 100, shuffle=True)
test_data = DataLoader(test, batch_size=1000, shuffle = False)

100%|██████████| 170M/170M [00:04<00:00, 35.7MB/s]


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#

#
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=(3,3),padding=(1,1)), #(3,32,32)>(32,32,32)
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=(3,3),padding=(1,1)), #(32,32,32)>(32,32,32)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(32,32,32)>(32,16,16)
            nn.Dropout(0.25),

            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=(3,3),padding=(1,1)), #(32,16,16)>(64,16,16)
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=(3,3),padding=(1,1)), #(64,16,16)>(64,16,16)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(64,16,16)>(64,8,8)
            nn.Dropout(0.25),

            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(3,3),padding=(1,1)), #(64,8,8)>(128,8,8)
            nn.MaxPool2d(kernel_size=2,stride=2), #(128,8,8)>(128,4,4)
            nn.ReLU(),
            nn.Dropout(0.25),

            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=(3,3),padding=(1,1)), #(128,4,4)>(128,4,4)
            nn.MaxPool2d(kernel_size=2,stride=2), #(128,4,4)>(128,2,2)
            nn.ReLU(),
            nn.Dropout(0.25),

            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=(3,3),padding=(1,1)), #(128,2,2)>(256,2,2)
            nn.MaxPool2d(kernel_size=2,stride=2), #(256,2,2)>(256,1,1)
            nn.ReLU(),
            nn.Dropout(0.25),
        )
        self.g = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10),
        )
    def forward(self, x) :
        x = self.f(x)
        x = x.reshape(-1, 256)
        x = self.g(x)
        return x

F = NN()
F = F.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(F.parameters(), lr = 0.001)
epoch = 200
cnt = 0
prev_acc = 0

F.train()
for e in tqdm(range(epoch)) :
    loss_sum = 0
    for x, t in train_data :
        x = x.to(device)
        t = t.to(device)
#
        y = F(x)
#
        loss = loss_func(y, t)
        loss_sum += loss
#
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    loss_sum /= len(train_data)

    # F.eval()
    # correct = 0
    # total = 0
    # for x, t in test_data :
    #     x = x.to(device)
    #     t = t.to(device)
    #     with torch.no_grad() :
    #         y = F(x)
    #     correct += (y.argmax(dim=1) == t).sum().item()
    #     total += len(x)
    # acc = correct / total
    # if acc <= prev_acc :
    #     cnt += 1
    # else :
    #     cnt = 0
    #     prev_acc = acc
    # print(f"epoch {e+1} | loss {loss_sum} | acc {acc}")
    # if cnt >= 3 :
    #     print("train halted")
    #     break
    # F.train()

100%|██████████| 200/200 [28:26<00:00,  8.53s/it]


In [14]:
F.eval()
correct = 0
total = 0
for x, t in test_data :
    x = x.to(device)
    t = t.to(device)
    with torch.no_grad() :
        y = F(x)
    correct += (y.argmax(dim=1) == t).sum().item()
    total += len(x)
acc = correct / total

print(f"acc {acc}")

acc 0.8266


In [12]:
F.eval()
correct = 0
total = 0
for x, t in train_data :
    x = x.to(device)
    t = t.to(device)
    with torch.no_grad() :
        y = F(x)
    correct += (y.argmax(dim=1) == t).sum().item()
    total += len(x)
acc = correct / total

print(f"acc {acc}")

acc 0.93622


In [13]:
torch.save(F.to(device), "CIFAR10.pt")

In [ ]:
print(train[0][0].shape) #x의 모양
print(min(train.targets)) #t최댓값
print(max(train.targets)) #t최솟값
print(train[0][0])

torch.Size([3, 32, 32])
0
9
tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
         [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
         [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
         ...,
         [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
         [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
         [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],

        [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
         [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
         [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
         ...,
         [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
         [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
         [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],

        [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
         [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
         [0.0824, 0.0000, 0.03